## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [2]:
# импортируем pandas
import pandas as pd

# читаем файл
data = pd.read_csv('/datasets/data.csv')

# первая проверка
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

В процессе первичной проверки данных было обнаружено следующее:
- Названия столбцов соответствуют требованиям анализа данных;
- В столбцах 'children' и 'days_employed' обнаружены отрицательные значения и выбросы;
- В столбце 'dob_years' обнаружены нулевые значения;
- В столбце 'education' обнаружены различные регистры;
- В столбцах 'days_employed' и 'total_income' обнаружены пропуски.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
# обработаем нули в столбце 'dob_years', заполнив средними значениями в зависимости от категорий 'income_type'
# обозначим функцию среднего возраста в зависимости от категории дохода
def age_according_to_income_type(row):
    if row['dob_years'] == 0:
        return round(data[data['income_type'] == row['income_type']]['dob_years'].mean())
    return round(row['dob_years'])

# применим функцию к датасету
data['dob_years'] = data.apply(age_according_to_income_type, axis=1) 

# далее необходимо обработать данные столбца 'days_employed'
# обозначим функцию приведения числа к модулю
def module(number): 
    if number > 0:
        return number
    return -number

# применим функцию к отрицательным значениям
data['days_employed'] = data['days_employed'].apply(module) 

# посчитаем среднее количество отработанных дней на каждый год жизни после 18 лет для данных без выбросов 
days_employed_mean_per_year = data[data['days_employed'] < 300000]['days_employed'].mean() / (data[data['days_employed'] < 300000]['dob_years'].mean() - 18)

# заменим выбросы и пропуски усредненными значениями, основываясь на возрасте
def normalyzer(row):
    if row['days_employed'] < 300000:
        return row['days_employed']   
    return days_employed_mean_per_year * (row['dob_years'] - 18)

# применим функцию к датасету 
data['days_employed'] = data.apply(normalyzer, axis=1) 

# очередь обработки столбца 'total_income'
# обработаем NaN в столбце 'total_income', заполнив медианными значениями

income_median = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(value = income_median)

# обработаем отрицательные значения и выбросы в столбце 'children'
# определим функцию, возвращающую корректные значения количества детей

def children_normal(children): 
    if children < 0:
        return -children
    if children >= 10:
        return children // 10
    return children

# применим функцию к датасету 
data['children'] = data['children'].apply(children_normal) 

# проверка
data.isna().sum()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

В процессе обработки пропусков было выполнено следующее:
- В первую очередь был обработан столбец 'dob_years', в котором нулевые значения были заменены средними значениями по категориям стобца 'income_type';
- Основной задачей была обработка отрицательных и выпадающих значений столбца 'days_employed'. Было выяснено, что категория пенсионеров отображается некорректно, и количество отработанных дней в переводе на года превышает возраст пенсионеров. Поэтому отрицательные значения были изменены на положительные, а выпадающие - заменены на произведение среднегодового количества отработанных дней после совершеннолетия на возраст после совершеннолетия;
- Пропущенные значения столбца 'total_income' были заменены на медианные значения, чтобы нивелировать влияние выбросов;
- Отрицательные значения и выборосы в столбце 'children' были заменены на логически верные.

### Замена типа данных

In [4]:
# заменим типы данных float на int
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

# проверка
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Вещественный тип данных столбцов 'days_employed' и 'total_income' нарушали наглядность данных и не приносили никакого дополнительного качества в датасет, поэтому было принято решение заменить вещественный тип данных на целочисленный методом 'astype()'

### Обработка дубликатов

In [5]:
data['education'] = data['education'].str.lower() # приводим данные в единый регистр

data = data.drop_duplicates().reset_index(drop=True) # избавляемся от дубликатов с восстановлением индексов

data.duplicated().sum() # проверяем наличие дубликатов

0

### Вывод

Для обработки дубликатов было выполнено следующее:
- Данные столбца 'education' были приведены в единый регистр методом str.lower();
- Методами drop_duplicates() и reset_index() были удалены дубликаты с восстановлением индексов.
Наличие дубликатов можно обосновать ошибкой выгрузки данных из различных источников, когда в датасет попали одинаковые данные из многих баз данных.

### Лемматизация

In [6]:
# импортируем pymystem
from pymystem3 import Mystem
m = Mystem()
lemmas = []
# импортируем счетчтик
from collections import Counter

# создаем цикл для лемматизации
for n in range(len(data['purpose'])):
    lemmas += m.lemmatize(data['purpose'][n])

# считаем наиболее употребляемые слова    
Counter(lemmas)

Counter({'покупка': 5897,
         ' ': 33570,
         'жилье': 4460,
         '\n': 21454,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 478,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'подержанный': 486,
         'получение': 1314,
         'высокий': 1374,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607})

### Вывод

Для лемматизации использовались библиотеки pymystem и collections. 
После анализа выводов стало ясно, что цели кредита можно разделить на 4 основные категории: 
- недвижимость;
- автомобили;
- свабьба;
- обучение.

### Категоризация данных

In [7]:
# определим функцию, укрупняющую причины до основных категорий
def categoryzator(purpose):
    if 'свадьб' in purpose:
        return 'свадьба'
    if 'недвижимост' in purpose:
        return 'недвижимость'
    if 'жиль' in purpose:
        return 'недвижимость'
    if 'автомобил' in purpose:
        return 'автомобиль'
    if 'образован' in purpose:
        return 'образование'
    return 'ошибка категоризации'

# применим функцию к датасету для категоризации
data['purpose_category'] = data['purpose'].apply(categoryzator)

# проверка
data['purpose_category'].value_counts()

# определим функцию, укрупняющую доход до четырех основных категорий
def income_categoryzator(income):
    if income < 80000:
        return 'низкого дохода'
    if 80000 <= income < 150000:
        return 'среднего дохода'
    if 150000 <= income < 300000:
        return 'высокого дохода'
    return 'богатые'

# применим функцию к датасету для категоризации
data['income_category'] = data['total_income'].apply(income_categoryzator)

# определим функцию, укрупняющую количество детей до трех основных категорий
def children_categoryzator(children):
    if children == 0:
        return 'заемщик без детей'
    if children == 1:
        return 'заемщик с одним ребенком'
    return 'многодетный заемщик'

# применим функцию к датасету для категоризации
data['children_category'] = data['children'].apply(children_categoryzator)

# проверка
data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 15 columns):
children             21454 non-null int64
days_employed        21454 non-null int64
dob_years            21454 non-null int64
education            21454 non-null object
education_id         21454 non-null int64
family_status        21454 non-null object
family_status_id     21454 non-null int64
gender               21454 non-null object
income_type          21454 non-null object
debt                 21454 non-null int64
total_income         21454 non-null int64
purpose              21454 non-null object
purpose_category     21454 non-null object
income_category      21454 non-null object
children_category    21454 non-null object
dtypes: int64(7), object(8)
memory usage: 2.5+ MB


### Вывод

Для ответа на вопросы из Шага 3 нам необходимо выделить категории по 4 характеристикам: семейное положение, количество детей, уровень дохода и цель кредита. Для первой характеристики категории были выделены достаточно четко без дополнительной обработки, для оставщихся было выполнено следующее: 
- Для категоризации целей кредита была проведена лемматизация, которая выявила 4 основные цели кредита: недвижимость, автомобили, свабьба, обучение. Поэтому было достаточно определить функцию, которая бы присваивала одну из этих категорий для каждого наблюдения.
- Для категоризации по уровню дохода были выделены 4 основные категории граждан: люди низкого дохода - до 80 тыс. руб. в месяц; люди среднего дохода - до 150 тыс. руб. в месяц; люди высокого дохода - до 300 тыс. руб. в месяц; богатые - свыше 300 тыс. руб. в месяц;
- Для категоризации по количеству детей были выделены три основные группы: заемщики без детей; заемщики с одним ребенком; заемщики с двумя или более детей.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [8]:
# создаем таблицу, отражающую зависимость неплатежа по кредиту от количества детей заемщика
children_debt = data.groupby('children_category').agg({'debt':['count', 'sum']})
children_debt['conversion'] = children_debt['debt']['sum'] / children_debt['debt']['count'] 

# визуализируем данные
children_debt.sort_values('conversion')

debt       conversion
                          count   sum           
children_category                               
заемщик без детей         14091  1063   0.075438
заемщик с одним ребенком   4855   445   0.091658
многодетный заемщик        2508   233   0.092903

### Вывод

В ходе проведенного анализа был получен вывод, что заемщики без детей возвращают кредит вовремя чаще, чем заемщики с 1 и более детей.
Возможно, причина в том, что дети оказывают дополнительную финансовую нагрузку на заемщиков, что негативно сказывается на своевременном погашении кредитов.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
# создаем таблицу, отражающую зависимость неплатежа по кредиту от количества детей заемщика
family_status_debt = data.groupby('family_status').agg({'debt':['count', 'sum']})
family_status_debt['conversion'] = family_status_debt['debt']['sum'] / family_status_debt['debt']['count'] 

# визуализируем данные
family_status_debt.sort_values('conversion')

debt      conversion
                       count  sum           
family_status                               
вдовец / вдова           959   63   0.065693
в разводе               1195   85   0.071130
женат / замужем        12339  931   0.075452
гражданский брак        4151  388   0.093471
Не женат / не замужем   2810  274   0.097509

### Вывод

В ходе анализа зависимости своевременных возвратов кредитов от семейного положения был сделан вывод, что заемщики, которые когда-либо были зарегестрированы в официальном браке, значительно чаще возвращают кредиты в срок.
Обосновать данные выводы можно тем, что в официальный брак зачастую вступают более ответственные и финансово стабильные граждане.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
# создаем таблицу, отражающую зависимость неплатежа по кредиту от количества детей заемщика
income_debt = data.groupby('income_category').agg({'debt':['count', 'sum']})
income_debt['conversion'] = income_debt['debt']['sum'] / income_debt['debt']['count'] 

# визуализируем данные
income_debt.sort_values('conversion')

debt      conversion
                count  sum           
income_category                      
богатые          1482  106   0.071525
низкого дохода   2276  174   0.076450
высокого дохода  7702  620   0.080499
среднего дохода  9994  841   0.084150

### Вывод

В ходе анализа были получены следующие выводы:
- Богатые, очевидно, значительно чаще возвращают кредит в срок. Это можно обосновать более вдумчивым подходом к финансам (собственно, не зря богатые люди богаты, верно?);
- Довольно неожиданно, но люди с низкими доходами справляются с возвратом кредитов лучше, чем люди со средним и высокими доходами. Возможно, причина в том, что люди с низкими доходами берут кредиты на более необходимые вещи, чем люди со средними и высокими доходами, зачастую пытающиеся казаться более обеспеченными, чем они есть на самом деле.

- Как разные цели кредита влияют на его возврат в срок?

In [11]:
# создаем таблицу, отражающую зависимость неплатежа по кредиту от количества детей заемщика
purpose_debt = data.groupby('purpose_category').agg({'debt':['count', 'sum']})
purpose_debt['conversion'] = purpose_debt['debt']['sum'] / purpose_debt['debt']['count'] 

# визуализируем данные
purpose_debt.sort_values('conversion')

debt      conversion
                  count  sum           
purpose_category                       
недвижимость      10811  782   0.072334
свадьба            2324  186   0.080034
образование        4013  370   0.092200
автомобиль         4306  403   0.093590

### Вывод

В ходе анализа зависимости возвратности кредита в срок от целей его получения, были достигнуты следующие результаты:
- Наиболее часто возвращают кредит в срок заемщики, которые покупают или ремонтируют недвижимость. Основная причина, обусловившая данный результат, кроектся в том, что ипотечное жилье являтеся залогом кредита, и зачастую является единственным жильем заемщика;
- Довольно неожиданный результат, но кредиты на свадьбу погашаются вовремя чаще, чем кредиты на образование. В качестве возможной причины можно привести тот факт, что кредиты на свадьбу зачастую являются краткосрочными, и быстро погашаются подаренными гостями деньгами (если, конечно, заемщики финансово грамотны и ответственны). Кредиты на обучение, наоборот, в силу специфики обучения, являются более долгосрочными и при этом зачастую не приносят увеличения доходов в будущем, как ожидает заемщик;
- Худшей статистикой возвратности обладают кредиты на покупку автомобиля. Возможная причина кроется в том, что заемщики не всегда понимают, что наличие автомобиля значительно повышает требования к доходам, так как автомобиль требует постоянных финансовых вложений (бензин, страховка, мойка, плановое ТО, замена масла).

### Шаг 4. Общий вывод

В ходе данного проекта получены следующие выводы:
- Да, семейное положение и наличие детей значительно влияет на факт погашения кредита в срок. Однако неожиданным выводом стал факт, что эти метрики обратнозависимые - женатые люди возвращают кредиты вовремя чаще, чем неженатые, но при этом заемщики с детьми возвращают кредиты вовремя реже, чем заемщики без детей. Данная зависимость требует дальнейшей проработки на более объемной выборке;
- Неожиданным выводом стало то, что кредиты на свадьбу, несмотря на свою неоднозначную репутацию в обществе, возвращаются вовремя чаще, чем кредиты на образование. Хотя основная причина, по моему мнению, в сроке кредита. Было бы интересно получить и данную метрику в следующий раз, чтобы провести более глубокий анализ;
- Вторым неожиданным выводом стало то, что люди с низкими доходами лучше справляются с своевременными возвратами кредитов, чем заемщики со средними и высокими доходами. Возможно, причина кроется в том, что людям с низкими доходами значительно чаще приходится планировать затраты на месяц, так как свободных денежных средств попросту нет. 

Есть уверенность, что данные метрики могут быть применены в разработке системы кредитного скоринга.